In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
import os
import torch
import torch.backends.cudnn as cudnn
import numpy as np
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
#from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms

In [0]:
import dlib
import cv2
import os
import re
import json
from pylab import *
from PIL import Image, ImageChops, ImageEnhance
from tqdm.notebook import tqdm

In [0]:
!pip install facenet_pytorch

     |████████████████████████████████| 1.9MB 1.4MB/s 


In [0]:
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face

In [0]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [0]:
#mtcnn = MTCNN(margin=40, keep_all=True, thresholds = [0.8,0.7,0.7], factor=0.5, device=device).eval()
mtcnn = MTCNN(margin=40, keep_all=True, factor=0.5, min_face_size = 80, device=device).eval()
resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()

In [0]:
resnet101 = models.resnet101(pretrained=True)
resnet101 = resnet101.to(device)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth


In [0]:
import glob
filenames = glob.glob('/content/drive/My Drive/DLProject/dfdc_train/dfdc_train_part_1/*.mp4')

In [0]:
filenames

In [0]:
len(filenames)

1699

In [0]:
filenames[1]

'/content/drive/My Drive/DLProject/dfdc_train/dfdc_train_part_1/eupisnxhsb.mp4'

In [0]:
vid_names = []
for i in range(len(filenames)):
  vid_names.append(filenames[i].split('/')[-1])

In [0]:
np.save('/content/drive/My Drive/DLProject/Real_filenames_resnet101.npy', vid_names)

In [0]:
from IPython.display import display

In [0]:
class DetectionPipeline:
    def __init__(self, detector, n_frames=None, batch_size=1):
        self.detector = detector
        self.n_frames = n_frames
        self.batch_size = batch_size
        self.resize = resize
    
    def __call__(self, filename):
        """Load frames from an MP4 video and detect faces.

        Arguments:
            filename {str} -- Path to video.
        """
        # Create video reader and find length
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        #print(v_len)

        # Pick 'n_frames' evenly spaced frames to sample
        if self.n_frames is None:
            sample = np.arange(0, v_len, 5)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

        # Loop through frames
        faces = []
        frames = []
        for j in range(v_len):
            success = v_cap.grab()
            if j in sample:
                # Load frame
                success, frame = v_cap.retrieve()
                if not success:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)
                
                
                # Resize frame to desired size
                # if self.resize is not None:
                #     frame = frame.resize((self.resize, self.resize))
                #     #frame = frame.resize([int(d * self.resize) for d in frame.size])
                #     #print(frame.size)
                frames.append(frame)
                #display(frame)
                #cv2.imshow(frame)

                # When batch is full, detect faces and reset frame list
                if len(frames) % self.batch_size == 0 or j == sample[-1]:
                    face_extract = self.detector(frames)
                    # if face_extract is not None:
                    #   face_extract = face_extract.resize((self.resize, self.resize))
                    faces.extend(face_extract)
                    frames = []

        v_cap.release()

        return faces    


def process_faces(faces, resnet):
    # Filter out frames without faces
    faces = [f for f in faces if f is not None]
    faces = torch.cat(faces).to(device)

    # Generate facial feature vectors using a pretrained model
    embeddings = resnet(faces)
    
    return embeddings

In [0]:
detection_pipeline = DetectionPipeline(detector=mtcnn,batch_size=60)

In [0]:
c = detection_pipeline(filenames[0])

In [0]:
c[1].shape

torch.Size([1, 3, 160, 160])

In [0]:
for i in range(len(c)):
  if c[i] is not None:
    print(c[i].shape)

torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1, 3, 160, 160])
torch.Size([1,

In [0]:
x = process_faces(c, resnet)

In [0]:
x.shape

torch.Size([60, 512])

In [0]:
for i in range(len(x)):
  print(x[i].shape)

torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size

In [0]:
filenames[2]

'/content/drive/My Drive/DLProject/dfdc_train/dfdc_train_part_1/lurzowfubv.mp4'

In [0]:
X = []
start = time.time()
n_processed = 0
with torch.no_grad():
    for i, filename in tqdm(enumerate(filenames), total=len(filenames)):
        try:
            # Load frames and find faces
            faces = detection_pipeline(filename)
            print(len(faces))

            #faces = [f for f in faces if f is not None]
            #faces = torch.cat(faces).to(device)
            
            # Calculate embeddings
            X.append(process_faces(faces, resnet101))
            #X.append(faces)

        except KeyboardInterrupt:
            print('\nStopped.')
            break

        except Exception as e:
            print(e)
            X.append(None)
        
        n_processed += len(faces)
        print(f'Frames per second (load+detect+embed): {n_processed / (time.time() - start):6.3}\r', end='')

60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
6

In [0]:
n_processed

63166

In [0]:
type(X)

list

In [0]:
len(X)

363

In [0]:
count = 0
for i in range(len(X)):
  print(X[i].shape)
  if X[i] is None:
    count = count+ 1
print(count)

torch.Size([106, 1000])
torch.Size([245, 1000])
torch.Size([143, 1000])
torch.Size([432, 1000])
torch.Size([245, 1000])
torch.Size([221, 1000])
torch.Size([168, 1000])
torch.Size([142, 1000])
torch.Size([220, 1000])
torch.Size([80, 1000])
torch.Size([190, 1000])
torch.Size([418, 1000])
torch.Size([354, 1000])
torch.Size([197, 1000])
torch.Size([311, 1000])
torch.Size([176, 1000])
torch.Size([203, 1000])
torch.Size([185, 1000])
torch.Size([175, 1000])
torch.Size([183, 1000])
torch.Size([87, 1000])
torch.Size([76, 1000])
torch.Size([290, 1000])
torch.Size([143, 1000])
torch.Size([411, 1000])
torch.Size([134, 1000])
torch.Size([140, 1000])
torch.Size([55, 1000])
torch.Size([188, 1000])
torch.Size([134, 1000])
torch.Size([122, 1000])
torch.Size([222, 1000])
torch.Size([177, 1000])
torch.Size([174, 1000])
torch.Size([177, 1000])
torch.Size([212, 1000])
torch.Size([161, 1000])
torch.Size([436, 1000])
torch.Size([179, 1000])
torch.Size([129, 1000])
torch.Size([181, 1000])
torch.Size([153, 100

In [0]:
import numpy as np
np.save('/content/drive/My Drive/DLProject/dfdc_real_dat_resnet101.npy',X)

In [0]:
import glob
fake_filenames = glob.glob('/content/drive/My Drive/DLProject/manipulated_sequences/DeepFakeDetection/c23/videos/*.mp4')

In [0]:
len(fake_filenames)

456

In [0]:
fake_filenames[0]

'/content/drive/My Drive/DLProject/manipulated_sequences/DeepFakeDetection/c23/videos/02_25__secret_conversation__Z7FQ69VP.mp4'

In [0]:
class fake_DetectionPipeline:
    def __init__(self, detector, n_frames=None, batch_size=1):
        self.detector = detector
        self.n_frames = n_frames
        self.batch_size = batch_size
        self.resize = resize
    
    def __call__(self, filename):
        """Load frames from an MP4 video and detect faces.

        Arguments:
            filename {str} -- Path to video.
        """
        # Create video reader and find length
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        #print(v_len)

        # Pick 'n_frames' evenly spaced frames to sample
        if self.n_frames is None:
            sample = np.arange(0, v_len, 5)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

        # Loop through frames
        faces = []
        frames = []
        for j in range(v_len):
            success = v_cap.grab()
            if j in sample:
                # Load frame
                success, frame = v_cap.retrieve()
                if not success:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)
                
                
                # Resize frame to desired size
                # if self.resize is not None:
                #     frame = frame.resize((self.resize, self.resize))
                #     #frame = frame.resize([int(d * self.resize) for d in frame.size])
                #     #print(frame.size)
                frames.append(frame)
                #display(frame)
                #cv2.imshow(frame)

                # When batch is full, detect faces and reset frame list
                if len(frames) % self.batch_size == 0 or j == sample[-1]:
                    face_extract = self.detector(frames)
                    # if face_extract is not None:
                    #   face_extract = face_extract.resize((self.resize, self.resize))
                    faces.extend(face_extract)
                    frames = []

        v_cap.release()

        return faces    


def process_faces(faces, resnet):
    # Filter out frames without faces
    faces = [f for f in faces if f is not None]
    faces = torch.cat(faces).to(device)

    # Generate facial feature vectors using a pretrained model
    embeddings = resnet(faces)
    
    return embeddings

In [0]:
fake_detection_pipeline = fake_DetectionPipeline(detector=mtcnn,batch_size=60)

In [0]:
fake_sample_test = fake_detection_pipeline(fake_filenames[0])

In [0]:
len(fake_sample_test)

8

In [0]:
f = process_faces(fake_sample_test, resnet101)

In [0]:
len(f)

16

In [0]:
fake_dat = []
start = time.time()
n_processed = 0
with torch.no_grad():
    for i, filename in tqdm(enumerate(fake_filenames), total=len(fake_filenames)):
        try:
            # Load frames and find faces
            faces = fake_detection_pipeline(filename)
            print(len(faces))

            # Calculate embeddings
            fake_dat.append(process_faces(faces, resnet101))

        except KeyboardInterrupt:
            print('\nStopped.')
            break

        except Exception as e:
            print(e)
            fake_dat.append(None)
        
        n_processed += len(faces)
        print(f'Frames per second (load+detect+embed): {n_processed / (time.time() - start):6.3}\r', end='')

8
28
32
1
4
7
12
10
2
4
9
12
6
2
8
7
4
8
9
3
13
6
7
1
2
13
1
14
3
13
21
3
13
8
3
4
4
26
13
9
4
9
3
243
81
199
93
182
163
176
143
169
156
154
104
73
47
48
108
127
146
115
128
168
104
170
154
97
73
168
159
155
148
154
174
143
104
148
52
172
164
130
180
157
168
118
56
29
160
131
81
291
113
77
62
191
168
259
107
171
112
222
140
168
168
102
158
173
247
73
183
254
291
176
101
130
173
134
90
108
134
137
254
265
95
149
180
225
81
173
258
165
226
145
143
104
166
57
171
184
110
185
59
138
209
173
127
305
172
195
133
228
133
243
215
285
133
97
168
66
81
116
112
67
190
195
184
153
185
205
119
146
149
197
302
84
157
172
143
205
285
165
183
84
172
138
173
276
85
174
168
131
172
103
207
182
171
83
284
182
191
96
179
133
140
218
136
46
151
160
72
195
173
174
283
145
138
139
69
231
195
138
67
175
157
136
216
137
174
146
189
132
179
52
70
168
96
293
170
176
78
143
215
59
152
49
77
218
314
72
115
145
85
220
59
169
324
154
192
59
147
147
48
232
29
65
128
159
177
214
97
215
256
39
169
169
175
215
108
107
1

In [0]:
len(fake_dat[0])

16

In [0]:
len(fake_dat)

456

In [0]:
count = 0
fake_vid_names = []
fake_dat_cleaned = []
for i in range(len(fake_dat)):
  if len(fake_dat[i]) > 50:
    fake_dat_cleaned.append(fake_dat[i])
    fake_vid_names.append(fake_filenames[i].split('/')[-1])
    count = count + 1
print(count)

405


In [0]:
len(fake_dat_cleaned)

405

In [0]:
len(fake_vid_names)

405

In [0]:
np.save('/content/drive/My Drive/DLProject/fake_dat_cleaned_resnet101.npy',fake_dat_cleaned)

In [0]:
np.save('/content/drive/My Drive/DLProject/fake_vid_names_resnet101.npy',fake_vid_names)